# Zestaw 1


### Zadanie 1

Polega na zamianie tekstu zakodowanego w hex (systemie szesnastkowym) na kodowanie base64 \
Przykład:


In [8]:
data = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"

Powinien zwrócić:\
**SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t**

In [9]:
def hex_to_base64(text):
    return b64encode(bytes.fromhex(text)).decode()
def base64_to_hex(text):
    return b64decode(text.encode()).hex()
print(hex_to_base64(data))

SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t


### Zadanie 2

Napisz funkcję xorująca napisy o tej samej długości

In [20]:
def xor(first: bytes, second: bytes) -> bytes:
    result = bytearray(len(first))
    for ind,(x,y) in enumerate(list(zip(first,second))):
        result[ind] = x ^ y
    return result
example1 = bytes.fromhex("1c0111001f010100061a024b53535009181c") 
example2 = bytes.fromhex("686974207468652062756c6c277320657965")

print(xor(example1,example2).decode())

the kid don't play


### Zadanie 3

Dostaliśmy informacje, że nasz szyfrogram to tekst zxorowany z jednym znakiem:

In [33]:
ciphertext = "1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736"
ciphertext = bytes.fromhex(ciphertext)

W dalszej części zadań, będę używać implementacji xora z pwntools

In [30]:
from pwn import xor

Teraz wystarczy, że sprawdzimy nasz szyfrogram z każdym możliwym znakiem. \
By nie przeglądać możliwych tekstów jawnych ręcznie, potrzebujemy funkcji, która będzie oceniała na ile dobry jest wynik.


In [45]:
freq = { ' ' : 1, 'e' : 1.3, 'E': 1.3, 'T' : 0.91, 't' : 0.91, 'A' : 0.82, 'a' : 0.82, 'O' : 0.75, 'o': 0.75,
         'I' : 0.7, 'i': 0.7,'N' : 0.67, 'n':0.67,'S' : 0.63, 's': 0.63, 'H' : 0.61, 'h': 0.61, 'R' : 0.6,
         'r': 0.6, 'D' : 0.43, 'd':0.43,'L' : 0.4 , 'l': 0.4 ,    'C' : 0.28 , 'c': 0.28 ,'U' : 0.28 , 'u': 0.28 ,
         'M' : 0.24 , 'm': 0.24 ,'W' : 0.24 , 'w': 0.24 , 'F' : 0.22 , 'f': 0.22 ,'G' : 0.2 , 'g': 0.2 ,
         'Y' : 0.2 , 'y': 0.2, 'P' : 0.19 , 'p': 0.19, 'B' : 0.15, 'b':0.15, 'V' : 0.098, 'v' : 0.098, 'K' : 0.077,
         'k' : 0.077, 'X' : 0.015, 'J': 0.015}
alphabet = b'ABCDEFGHIJKLMNOPRSTUVWXY ' 

def getScore(text):
    score = 0
    text = text.upper()
    length = len(text)
    if length == 0:
        return 0
    for letter in alphabet:
        score += round(10*freq[chr(letter)] *(text.count(letter)/length),3)
    return round(score,2)

In [62]:
plaintexts = []
for character in range(ord('A'),ord('Z')):
    possible_plaintext = xor(character, ciphertext)
    plaintexts.append((getScore(possible_plaintext),possible_plaintext))
plaintexts = sorted(plaintexts, reverse=True)
for ind,(_,option) in enumerate(plaintexts[:3],1):
    print(f"{ind} opcja to {option.decode()}")

1 opcja to Cooking MC's like a pound of bacon
2 opcja to Ieeacdm*GI-y*fcao*k*zedn*el*hkied
3 opcja to Bnnjhof!LB&r!mhjd!`!qntoe!ng!c`bno


Funkcja, która przyda się do nastepnej części. Zwraca jedynie jeden najbardziej prawdopodobny napis.

In [67]:
def DecryptSingleByte(ciphertext: str) -> tuple:
    plaintexts = []
    for character in range(128):
        possible_plaintext = xor(character, ciphertext)
        plaintexts.append((getScore(possible_plaintext),possible_plaintext))
    plaintexts = sorted(plaintexts, reverse=True)
    return plaintexts[0]